In [ ]:
# install required libraries here

!pip install kaggle fastai fastkaggle pandas numpy

In [16]:
from fastkaggle import *
from google.colab import files

from fastai.data.all import *
from fastai.vision.all import *
from fastai.vision.widgets import *

In [ ]:
files.upload()

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [20]:
comp = 'nfl-player-contact-detection'

path = setup_comp(comp, install='fastai "timm>=0.6.2.dev0"')

100%|██████████| 3.84G/3.84G [00:34<00:00, 119MB/s]


In [21]:
path.ls()

(#10) [Path('nfl-player-contact-detection/train_player_tracking.csv'),Path('nfl-player-contact-detection/train'),Path('nfl-player-contact-detection/test_baseline_helmets.csv'),Path('nfl-player-contact-detection/train_labels.csv'),Path('nfl-player-contact-detection/test_video_metadata.csv'),Path('nfl-player-contact-detection/train_video_metadata.csv'),Path('nfl-player-contact-detection/sample_submission.csv'),Path('nfl-player-contact-detection/test_player_tracking.csv'),Path('nfl-player-contact-detection/train_baseline_helmets.csv'),Path('nfl-player-contact-detection/test')]